In [7]:
import sys
!{sys.executable} -m pip install holidays  pandas_market_calendars pika
!{sys.executable} -m pip install install --extra-index http://pypi.dcai2.local --upgrade --trusted-host pypi.dcai2.local dcaitrading=='0.0.6-29-g75d567f'

from datetime import date, timedelta

from featureeng.times import get_trading_times_for_london_ny
from featureeng.features import *
import featureeng as feng
import data.readers as rd

import multiprocessing
import time
import threading
import json
import os

import pyarrow as pa
import pyarrow.parquet as pq


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 4.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, http://pypi.dcai2.local


In [2]:


def prep_raw_data(instrument, year=None, get_ticks=False):
    breaks=['1Min', '5Min', '10Min', '15Min', '30Min', '1H', '2H', '4H', '6H', '12H', '1D']



    start_time = time.time()
    if get_ticks:
        ticks_for_instrument = rd.read_in_ticks('/home/dcai/data', instrument, year = year)
    else:
        ticks_for_instrument = None

    candles_dict = dict()
    for br in breaks:
        candles_dict[br] = rd.read_in_candles('/home/dcai/data/candles-right', instrument, br, year = year)

    end_time = time.time()
    print(f'Execution time: {end_time - start_time:.3f} seconds')


    start_date = date(2018, 4, 15)
    end_date = date(2022, 12, 29)

    trading_times = get_trading_times_for_london_ny(start_date=start_date, end_date=end_date)
    return breaks, ticks_for_instrument, candles_dict, trading_times


instrument = 'EURCHF'
breaks, ticks_for_instrument, candles_dict, trading_times = prep_raw_data(instrument, get_ticks=True)

EURCHF - /home/dcai/data
/home/dcai/data/EURCHF
EURCHF - /home/dcai/data/candles-right
EURCHF - /home/dcai/data/candles-right
EURCHF - /home/dcai/data/candles-right
EURCHF - /home/dcai/data/candles-right
EURCHF - /home/dcai/data/candles-right
EURCHF - /home/dcai/data/candles-right
EURCHF - /home/dcai/data/candles-right
EURCHF - /home/dcai/data/candles-right
EURCHF - /home/dcai/data/candles-right
EURCHF - /home/dcai/data/candles-right
EURCHF - /home/dcai/data/candles-right
Execution time: 17.456 seconds


In [3]:


feature_list=['close_MIN_1D_5',
#'close_MIN_1D_20',
#'close_MIN_1D_60',
'close_MAX_1D_5',
'close_MAX_1D_20',
#'close_MAX_1D_60',
'close_AVG_1D_5',
'close_AVG_1D_20',
#'close_AVG_1D_60',
'close_STDEV_1D_5',
'close_STDEV_1D_20',
#'close_STDEV_1D_60',
#'close_MEDIAN_1D_5',
#'close_MEDIAN_1D_20',
#'close_MEDIAN_1D_60',
'close_MIN_1H_5',
'close_MIN_1H_20',
'close_MIN_1H_60',
'close_MAX_1H_5',
'close_MAX_1H_20',
#'close_MAX_1H_60',
'close_AVG_1H_5',
'close_AVG_1H_20',
'close_AVG_1H_60',
'close_STDEV_1H_5',
'close_STDEV_1H_20',
#'close_STDEV_1H_60',
'close_MEDIAN_1H_5',
'close_MEDIAN_1H_20',
#'close_MEDIAN_1H_60',
'close_MIN_15Min_5',
'close_MIN_15Min_20',
'close_MIN_15Min_60',
'close_MAX_15Min_5',
'close_MAX_15Min_20',
'close_MAX_15Min_60',
'close_AVG_15Min_5',
'close_AVG_15Min_20',
'close_AVG_15Min_60',
'close_STDEV_15Min_5',
'close_STDEV_15Min_20',
'close_STDEV_15Min_60',
'close_MEDIAN_15Min_5',
'close_MEDIAN_15Min_20',
'close_MEDIAN_15Min_60']

are_trading_feats = ['London', 'NY']

ma_feats = ['rolling_15Min_20_3', 'ewm_1D_60_3','rolling_5Min_20_4', 'ewm_15Min_20_4','rolling_4H_20_4', 'ewm_4H_20_4']

ma_div_ma_feats = ['rolling_15Min_60_20_4','rolling_5Min_60_20_4','rolling_4H_60_20_4']

all_features = {
    'simple': feature_list,
    'trading_regions': are_trading_feats,
    'ma': ma_feats,
    'ma_div_ma': ma_div_ma_feats,
    'labels': ['label_120_0', 'label_180_0', 'label_60_0'],
    'instrument': instrument
}

In [4]:
def parquet_file_to_pd(file_path):
    return pq.read_table(file_path).to_pandas()

def pd_to_parquet(df, file_path):
    pq.write_table(pa.Table.from_pandas(df), file_path)

def process_features_for_day(all_features, candles_dict, ticks_for_day, sample_frac=None, sample_size=None, save_dir=None, feat_date=None):
    if save_dir and feat_date:
        year = feat_date.strftime('%Y')
        doy = feat_date.strftime("%j")
        result_dir = os.path.join(save_dir, year)
        
        result_file = os.path.join(result_dir, "{}.parquet".format(doy))
        if os.path.exists(result_file):
            print("file {} exists, skip".format(result_file))
            return
    
    
    res = calc_features_for_day(feature_list=all_features, 
                      candles_dict=candles_dict, 
                      ticks_for_day=ticks_for_day, 
                      sample_frac=sample_frac,
                      sample_size=sample_size)
    
    if save_dir and feat_date:
        year = feat_date.strftime('%Y')
        doy = feat_date.strftime("%j")
        
        result_dir = os.path.join(save_dir, year)
        result_file = os.path.join(result_dir, "{}.parquet".format(doy))
        if not os.path.exists(result_dir):
            os.makedirs(result_dir)
            
        pq.write_table(pa.Table.from_pandas(res), result_file)

def process_features_for_day_from_file(all_features, candles_dict, ticks_for_day_file, sample_frac=None, sample_size=None, save_dir=None, feat_date=None):

    ticks_for_day = parquet_file_to_pd(ticks_for_day_file)
    print("Doing {}".format(ticks_for_day_file))
    start_time = time.time()
    process_features_for_day(all_features, candles_dict, ticks_for_day, sample_frac, sample_size, save_dir, feat_date)
    end_time = time.time()
    print(f'Done {ticks_for_day_file} in {end_time - start_time:.3f} seconds')
    

def extract_year_and_file(file_path):
    parts = file_path.split('/')

    return parts[-2],parts[-1]

def enrich_features_for_day_from_file(org_feat_file, enrich_set, candles_dict, ticks_root_dir, tl_result_dir):

    year, file = extract_year_and_file(org_feat_file)
    
    result_dir = os.path.join(tl_result_dir, year)
    
    ticks_file_path = os.path.join(ticks_root_dir, year, file)
    output_file_path = os.path.join(result_dir, file)
    
    if os.path.exists(output_file_path):
        print("file {} exists, skip".format(output_file_path))
        return
    
    print("Doing {}".format(output_file_path))
    #given org_feat_file such as /home/dcai/data/features/EURCHF/1/2018/180.parquet
    org_feats = parquet_file_to_pd(org_feat_file)
    
    start_time = time.time()
    ticks_for_day = parquet_file_to_pd(ticks_file_path)

    
    new_feats = calc_features_for_day(feature_list=enrich_set, 
                      candles_dict=candles_dict, 
                      ticks_for_day=ticks_for_day, 
                      selector_index=org_feats.index)
    

    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
    
    
    res = pd.merge(org_feats, new_feats, left_index=True, right_index=True, how='inner')
    
    pq.write_table(pa.Table.from_pandas(res), output_file_path)
    
    end_time = time.time()
    print(f'Done {output_file_path} in {end_time - start_time:.3f} seconds')

    

In [13]:
org_feat_file = '/home/dcai/data/features/EURCHF/1/2021/139.parquet'
year, file = extract_year_and_file(org_feat_file)
ticks_root_dir = "/home/dcai/data/features/{}/ticks-for-day".format(instrument)
ticks_file_path = os.path.join(ticks_root_dir, year, file)

org_feats = parquet_file_to_pd(org_feat_file)
ticks_for_day = parquet_file_to_pd(ticks_file_path)

new_feats = calc_features_for_day(feature_list=enrich_set, 
                      candles_dict=candles_dict, 
                      ticks_for_day=ticks_for_day, 
                      selector_index=org_feats.index)



res = pd.merge(org_feats, new_feats, left_index=True, right_index=True, how='inner')


print(res.head(10))

print("----------------------")

print(res.info())

Using selector index
                         MeanPrice_x  close_MIN_1D_5  close_MIN_1D_20  \
DateTime                                                                
2021-05-19 07:00:00.041     1.098170        0.996776         0.995570   
2021-05-19 07:00:00.301     1.098155        0.996790         0.995584   
2021-05-19 07:00:00.527     1.098180        0.996767         0.995561   
2021-05-19 07:00:00.557     1.098165        0.996781         0.995574   
2021-05-19 07:00:00.812     1.098230        0.996722         0.995516   
2021-05-19 07:00:00.997     1.098290        0.996668         0.995461   
2021-05-19 07:00:01.184     1.098345        0.996618         0.995411   
2021-05-19 07:00:01.916     1.098315        0.996645         0.995438   
2021-05-19 07:00:03.576     1.098305        0.996654         0.995448   
2021-05-19 07:00:04.137     1.098270        0.996686         0.995479   

                         close_MIN_1D_60  close_MAX_1D_5  close_MAX_1D_20  \
DateTime                 

In [5]:
def prep_day_ticks(trading_days, ticks, save_dir):
    save_dir = os.path.join(save_dir, 'ticks-for-day')
    
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    for index, row in trading_days.iterrows():
        start = row['market_open_london']
        end = row['market_close_ny']
            
        ticks_for_day = ticks[start.strftime('%Y-%m-%d %H:%M:%S'):end.strftime('%Y-%m-%d %H:%M:%S')]
        if len(ticks_for_day)<1:
            continue
        

        year = start.strftime('%Y')
        doy = start.strftime("%j")
        
        result_dir = os.path.join(save_dir, year)
        
        if not os.path.exists(result_dir):
            os.makedirs(result_dir)
        
        result_file = os.path.join(result_dir, "{}.parquet".format(doy))
        pq.write_table(pa.Table.from_pandas(ticks_for_day), result_file)
                

def prep_features2(all_features, trading_days, ticks_root_dir, result_dir, fraction=None, sample_size=None, pool_size=20):
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
    
    with open(os.path.join(result_dir, 'feats.json'), 'w') as f:
        json.dump(all_features, f)
        
    no_trading_days = len(trading_days)
    
    
    
    group_size = 2000
    for i in range(0, no_trading_days, group_size):
        group = trading_days[i:i+group_size]
        args = []
        for index, row in group.iterrows():
            start = row['market_open_london']
            end = row['market_close_ny']

            year = start.strftime('%Y')
            doy = start.strftime("%j")

            ticks_file_dir = os.path.join(ticks_root_dir, year)
            ticks_file = os.path.join(ticks_file_dir, "{}.parquet".format(doy))

            if not os.path.exists(ticks_file):
                print("No ticks file for {}".format(start))
                continue
            args.append((all_features, candles_dict, ticks_file, fraction, sample_size, result_dir, start))
            
        print("doing group {}/{}".format(i, no_trading_days))
        with multiprocessing.Pool(pool_size) as pool:
            pool.starmap(process_features_for_day_from_file, args)


            
def load_feat_json(dir_path):
    with open(os.path.join(dir_path, 'feats.json'), "r") as file:
        return json.load(file)
    
def discover_feature_files(features_to_enrich_dir):
    filenames = []
    for root, dirs, files in os.walk(features_to_enrich_dir):
        for file in files:
            # Check if the file is a CSV file
            if file.endswith('.parquet'):
                filenames.append(os.path.join(root, file))
    return filenames

def enrich_features(enrich_set, ticks_root_dir, features_to_enrich_dir, result_dir, candles_dict, pool_size=16):
    #load original feat set, add enrichment and save as json
    org_feats = load_feat_json(features_to_enrich_dir)
    
    full_feats_set = org_feats | enrich_set
    
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
    
    with open(os.path.join(result_dir, 'feats.json'), 'w') as f:
        json.dump(full_feats_set, f)
    
    #iterate over files in original set
    org_feat_files = discover_feature_files(features_to_enrich_dir)
    args = []
    for org_feat_file in org_feat_files:
        args.append((org_feat_file, enrich_set, candles_dict, ticks_root_dir, result_dir))
        
    with multiprocessing.Pool(pool_size) as pool:
        pool.starmap(enrich_features_for_day_from_file, args)


In [ ]:


start_time = time.time()   
prep_day_ticks(trading_times, ticks_for_instrument, os.path.join('/home/dcai/data/features/', instrument))

end_time = time.time()
print(f'Execution time: {end_time - start_time:.3f} seconds')

In [ ]:
ticks_for_instrument = None

In [16]:

start_time = time.time()         

prep_features2(all_features, trading_times, 
               "/home/dcai/data/features/{}/ticks-for-day".format(instrument), 
               "/home/dcai/data/features/{}/5".format(instrument), 
               fraction=None, sample_size=20, pool_size=30)

end_time = time.time()
print(f'Execution time: {end_time - start_time:.3f} seconds')

doing group 0/1163
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/106.parquet
file /home/dcai/data/features/EURCHF/5/2018/106.parquet exists, skip
Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/106.parquet in 0.001 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/107.parquet
file /home/dcai/data/features/EURCHF/5/2018/107.parquet exists, skip
Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/107.parquet in 0.000 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/108.parquet
file /home/dcai/data/features/EURCHF/5/2018/108.parquet exists, skip
Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/108.parquet in 0.000 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/109.parquet
file /home/dcai/data/features/EURCHF/5/2018/109.parquet exists, skip
Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/109.parquet in 0.000 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/110.parquet
file /home/dcai/data

Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/158.parquet in 0.000 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/159.parquet
file /home/dcai/data/features/EURCHF/5/2018/159.parquet exists, skip
Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/159.parquet in 0.000 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/162.parquet
file /home/dcai/data/features/EURCHF/5/2018/162.parquet exists, skip
Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/162.parquet in 0.000 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/163.parquet
file /home/dcai/data/features/EURCHF/5/2018/163.parquet exists, skip
Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/163.parquet in 0.000 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/164.parquet
file /home/dcai/data/features/EURCHF/5/2018/164.parquet exists, skip
Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/164.parquet in 0.001 seconds
Doing /home/dcai/data/f

file /home/dcai/data/features/EURCHF/5/2018/212.parquet exists, skip
Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/212.parquet in 0.000 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/213.parquet
file /home/dcai/data/features/EURCHF/5/2018/213.parquet exists, skip
Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/213.parquet in 0.000 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/214.parquet
file /home/dcai/data/features/EURCHF/5/2018/214.parquet exists, skip
Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/214.parquet in 0.000 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/215.parquet
file /home/dcai/data/features/EURCHF/5/2018/215.parquet exists, skip
Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/215.parquet in 0.000 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/218.parquet
file /home/dcai/data/features/EURCHF/5/2018/218.parquet exists, skip
Done /home/dcai/data/features/EURCHF/ti

Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/267.parquet
file /home/dcai/data/features/EURCHF/5/2018/267.parquet exists, skip
Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/267.parquet in 0.001 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/268.parquet
file /home/dcai/data/features/EURCHF/5/2018/268.parquet exists, skip
Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/268.parquet in 0.000 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/269.parquet
file /home/dcai/data/features/EURCHF/5/2018/269.parquet exists, skip
Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/269.parquet in 0.000 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/270.parquet
file /home/dcai/data/features/EURCHF/5/2018/270.parquet exists, skip
Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/270.parquet in 0.001 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/271.parquet
file /home/dcai/data/features/EURCHF/5/

Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/317.parquet in 0.000 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/318.parquet
file /home/dcai/data/features/EURCHF/5/2018/318.parquet exists, skip
Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/318.parquet in 0.000 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/319.parquet
file /home/dcai/data/features/EURCHF/5/2018/319.parquet exists, skip
Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/319.parquet in 0.000 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/320.parquet
file /home/dcai/data/features/EURCHF/5/2018/320.parquet exists, skip
Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/320.parquet in 0.000 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2018/323.parquet
file /home/dcai/data/features/EURCHF/5/2018/323.parquet exists, skip
Done /home/dcai/data/features/EURCHF/ticks-for-day/2018/323.parquet in 0.001 seconds
Doing /home/dcai/data/f

...

Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/168.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2021/230.parquet in 2913.464 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/272.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2021/245.parquet in 2953.339 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/286.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/152.parquet in 3874.748 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/157.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/006.parquet in 3288.739 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/007.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2021/216.parquet in 3080.907 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/300.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/063.parquet in 3919.295 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/0

Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/082.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/010.parquet in 3371.519 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/011.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/243.parquet in 4472.412 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/244.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/329.parquet in 3230.655 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/332.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2021/316.parquet in 3198.642 seconds
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/139.parquet in 4207.467 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/140.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/025.parquet in 3358.792 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/026.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/20

Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/055.parquet in 3524.445 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/056.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/027.parquet in 2782.928 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/028.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/290.parquet in 3485.564 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/291.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/276.parquet in 3930.868 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/277.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/143.parquet in 3176.183 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/144.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/097.parquet in 2956.714 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/098.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/04

Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/292.parquet in 3029.015 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/293.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/117.parquet in 2853.306 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/118.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/131.parquet in 2897.573 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/132.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/102.parquet in 2737.924 seconds
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/250.parquet in 3112.347 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/251.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/278.parquet in 3493.475 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/279.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/164.parquet in 2867.248 seconds
Doing /home/dcai/data/features/EURCHF/t

Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/210.parquet in 1750.364 seconds
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/224.parquet in 1613.617 seconds
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/181.parquet in 1748.870 seconds
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/283.parquet in 1561.102 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/284.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/312.parquet in 1599.391 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/313.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/297.parquet in 1980.842 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/298.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/340.parquet in 1404.324 seconds
Doing /home/dcai/data/features/EURCHF/ticks-for-day/2022/341.parquet
Done /home/dcai/data/features/EURCHF/ticks-for-day/2022/196.parquet in 1556.772 seconds
Done /home/dcai/data

In [23]:

start_time = time.time()         

enrich_set = {
    'labels': ['label_60_5', 'label_120_5', 'label_180_5'],
    'instrument': instrument
}

ticks_root_dir = "/home/dcai/data/features/{}/ticks-for-day".format(instrument)
features_to_enrich_dir = "/home/dcai/data/features/{}/1".format(instrument)
result_dir = "/home/dcai/data/features/{}/2".format(instrument)

enrich_features(enrich_set=enrich_set, 
                ticks_root_dir=ticks_root_dir, 
                features_to_enrich_dir=features_to_enrich_dir,
                result_dir=result_dir, 
                candles_dict=candles_dict)

end_time = time.time()
print(f'Execution time: {end_time - start_time:.3f} seconds')

Doing /home/dcai/data/features/GBPUSD/2/2020/335.parquet
Using selector index
Doing /home/dcai/data/features/GBPUSD/2/2020/205.parquet
Using selector index
Doing /home/dcai/data/features/GBPUSD/2/2020/204.parquet
Using selector index
Doing /home/dcai/data/features/GBPUSD/2/2020/304.parquet
Using selector index
Doing /home/dcai/data/features/GBPUSD/2/2020/350.parquet
Using selector index
Doing /home/dcai/data/features/GBPUSD/2/2020/255.parquet
Using selector index
Doing /home/dcai/data/features/GBPUSD/2/2020/108.parquet
Using selector index
Doing /home/dcai/data/features/GBPUSD/2/2020/177.parquet
Using selector index
Doing /home/dcai/data/features/GBPUSD/2/2020/171.parquet
Using selector index
Doing /home/dcai/data/features/GBPUSD/2/2020/262.parquet
Using selector index
Doing /home/dcai/data/features/GBPUSD/2/2020/143.parquet
Using selector index
Doing /home/dcai/data/features/GBPUSD/2/2020/248.parquet
Using selector index
Doing /home/dcai/data/features/GBPUSD/2/2020/071.parquet
Using s

Done /home/dcai/data/features/GBPUSD/2/2020/073.parquet in 61.855 seconds
Doing /home/dcai/data/features/GBPUSD/2/2020/252.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2020/366.parquet in 16.271 seconds
Doing /home/dcai/data/features/GBPUSD/2/2020/349.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2020/253.parquet in 36.504 seconds
Doing /home/dcai/data/features/GBPUSD/2/2020/111.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2020/332.parquet in 19.801 seconds
Doing /home/dcai/data/features/GBPUSD/2/2020/118.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2020/261.parquet in 40.629 seconds
Doing /home/dcai/data/features/GBPUSD/2/2020/157.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2020/087.parquet in 48.117 seconds
Doing /home/dcai/data/features/GBPUSD/2/2020/156.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2020/202.parquet in 16.843 seconds
Doing /home/dc

Doing /home/dcai/data/features/GBPUSD/2/2020/329.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2020/259.parquet in 29.220 seconds
Doing /home/dcai/data/features/GBPUSD/2/2020/160.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2020/023.parquet in 24.677 seconds
Doing /home/dcai/data/features/GBPUSD/2/2020/006.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2018/107.parquet in 32.292 seconds
Doing /home/dcai/data/features/GBPUSD/2/2018/180.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2020/344.parquet in 18.226 seconds
Doing /home/dcai/data/features/GBPUSD/2/2020/114.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2020/010.parquet in 14.170 seconds
Doing /home/dcai/data/features/GBPUSD/2/2020/007.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2018/242.parquet in 22.115 seconds
Doing /home/dcai/data/features/GBPUSD/2/2018/130.parquet
Done /home/dcai/data/features/G

Using selector index
Done /home/dcai/data/features/GBPUSD/2/2018/178.parquet in 23.720 seconds
Doing /home/dcai/data/features/GBPUSD/2/2018/201.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2019/331.parquet in 13.974 seconds
Doing /home/dcai/data/features/GBPUSD/2/2019/081.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2019/193.parquet in 14.964 seconds
Doing /home/dcai/data/features/GBPUSD/2/2019/305.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2018/240.parquet in 20.399 seconds
Doing /home/dcai/data/features/GBPUSD/2/2018/284.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2019/196.parquet in 12.139 seconds
Doing /home/dcai/data/features/GBPUSD/2/2019/255.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2020/287.parquet in 22.316 seconds
Doing /home/dcai/data/features/GBPUSD/2/2020/209.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2019/093.parquet in 51.202 s

Doing /home/dcai/data/features/GBPUSD/2/2018/262.parquet
Using selector index
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2018/228.parquet in 17.979 seconds
Doing /home/dcai/data/features/GBPUSD/2/2018/108.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2019/052.parquet in 29.108 seconds
Doing /home/dcai/data/features/GBPUSD/2/2019/274.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2019/115.parquet in 22.598 seconds
Doing /home/dcai/data/features/GBPUSD/2/2019/079.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2018/234.parquet in 15.047 seconds
Doing /home/dcai/data/features/GBPUSD/2/2018/145.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2018/297.parquet in 38.869 seconds
Doing /home/dcai/data/features/GBPUSD/2/2018/317.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2018/143.parquet in 35.199 seconds
Doing /home/dcai/data/features/GBPUSD/2/2018/155.parquet
Using sele

...

Done /home/dcai/data/features/GBPUSD/2/2022/224.parquet in 20.047 seconds
Doing /home/dcai/data/features/GBPUSD/2/2022/189.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2022/181.parquet in 38.214 seconds
Doing /home/dcai/data/features/GBPUSD/2/2022/024.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2022/123.parquet in 14.123 seconds
Doing /home/dcai/data/features/GBPUSD/2/2022/186.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2022/028.parquet in 30.344 seconds
Doing /home/dcai/data/features/GBPUSD/2/2022/279.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2019/010.parquet in 25.471 seconds
Doing /home/dcai/data/features/GBPUSD/2/2019/007.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2019/179.parquet in 20.982 seconds
Doing /home/dcai/data/features/GBPUSD/2/2019/309.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2019/158.parquet in 19.169 seconds
Doing /home/dc

Doing /home/dcai/data/features/GBPUSD/2/2022/263.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2022/252.parquet in 40.138 seconds
Doing /home/dcai/data/features/GBPUSD/2/2022/251.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2019/296.parquet in 16.541 seconds
Doing /home/dcai/data/features/GBPUSD/2/2019/168.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2019/198.parquet in 13.313 seconds
Doing /home/dcai/data/features/GBPUSD/2/2019/330.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2019/213.parquet in 37.107 seconds
Doing /home/dcai/data/features/GBPUSD/2/2019/361.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2022/215.parquet in 21.797 seconds
Doing /home/dcai/data/features/GBPUSD/2/2022/039.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2022/234.parquet in 16.213 seconds
Doing /home/dcai/data/features/GBPUSD/2/2022/145.parquet
Done /home/dcai/data/features/G

Done /home/dcai/data/features/GBPUSD/2/2022/278.parquet in 39.393 seconds
Doing /home/dcai/data/features/GBPUSD/2/2022/124.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2021/126.parquet in 13.503 seconds
Doing /home/dcai/data/features/GBPUSD/2/2021/274.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2022/307.parquet in 56.676 seconds
Doing /home/dcai/data/features/GBPUSD/2/2022/147.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2021/034.parquet in 14.685 seconds
Doing /home/dcai/data/features/GBPUSD/2/2021/328.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2022/220.parquet in 17.364 seconds
Doing /home/dcai/data/features/GBPUSD/2/2022/249.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2021/204.parquet in 13.550 seconds
Doing /home/dcai/data/features/GBPUSD/2/2021/188.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2022/353.parquet in 46.056 seconds
Doing /home/dc

Doing /home/dcai/data/features/GBPUSD/2/2022/007.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2021/263.parquet in 15.928 seconds
Doing /home/dcai/data/features/GBPUSD/2/2021/120.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2021/319.parquet in 7.741 seconds
Doing /home/dcai/data/features/GBPUSD/2/2021/267.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2021/333.parquet in 9.154 seconds
Doing /home/dcai/data/features/GBPUSD/2/2021/144.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2021/116.parquet in 8.445 seconds
Doing /home/dcai/data/features/GBPUSD/2/2021/203.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2021/229.parquet in 6.802 seconds
Doing /home/dcai/data/features/GBPUSD/2/2021/344.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2022/201.parquet in 24.801 seconds
Doing /home/dcai/data/features/GBPUSD/2/2022/256.parquet
Using selector index
Done /home/dca

Done /home/dcai/data/features/GBPUSD/2/2021/197.parquet in 9.135 seconds
Doing /home/dcai/data/features/GBPUSD/2/2021/117.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2021/243.parquet in 13.753 seconds
Doing /home/dcai/data/features/GBPUSD/2/2021/271.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2022/137.parquet in 25.521 seconds
Doing /home/dcai/data/features/GBPUSD/2/2022/192.parquet
Done /home/dcai/data/features/GBPUSD/2/2021/238.parquet in 5.209 seconds
Doing /home/dcai/data/features/GBPUSD/2/2021/343.parquet
Using selector index
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2022/223.parquet in 23.726 seconds
Done /home/dcai/data/features/GBPUSD/2/2021/104.parquet in 9.213 seconds
Doing /home/dcai/data/features/GBPUSD/2/2021/337.parquet
Using selector index
Done /home/dcai/data/features/GBPUSD/2/2021/083.parquet in 9.445 seconds
Doing /home/dcai/data/features/GBPUSD/2/2021/350.parquet
Using selector index
Done /home/dcai/da

In [ ]:
table = pq.read_table('/home/dcai/data/features/EURCHF/ticks-for-day/2018/180.parquet')

ticks = table.to_pandas()

table = pq.read_table('/home/dcai/data/features/EURCHF/1/2018/180.parquet')
feats = table.to_pandas()

In [ ]:
#print(ticks.info())
#print(feats.info())



print(ticks.loc[ticks.index.intersection(feats.index)].info())

In [17]:
df2 = parquet_file_to_pd('/home/dcai/data/features/EURCHF/2/2021/139.parquet')
df1 = parquet_file_to_pd('/home/dcai/data/features/EURCHF/1/2021/139.parquet')

#print(df2.info())
#print(df1.info())

print(df2.info())
#print(df2.tail(10))


print(df1.info())
#print(df1.tail(10))



<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13785 entries, 2021-05-19 07:00:00.041000 to 2021-05-19 20:59:56.507000
Data columns (total 77 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   MeanPrice_x           13785 non-null  float64
 1   close_MIN_1D_5        13785 non-null  float64
 2   close_MIN_1D_20       13785 non-null  float64
 3   close_MIN_1D_60       13785 non-null  float64
 4   close_MAX_1D_5        13785 non-null  float64
 5   close_MAX_1D_20       13785 non-null  float64
 6   close_MAX_1D_60       13785 non-null  float64
 7   close_AVG_1D_5        13785 non-null  float64
 8   close_AVG_1D_20       13785 non-null  float64
 9   close_AVG_1D_60       13785 non-null  float64
 10  close_STDEV_1D_5      13785 non-null  float64
 11  close_STDEV_1D_20     13785 non-null  float64
 12  close_STDEV_1D_60     13785 non-null  float64
 13  close_MEDIAN_1D_5     13785 non-null  float64
 14  close_MEDIAN_1D_20   

In [12]:
import pandas as pd

df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                    'B': ['B0', 'B1', 'B2', 'B3']},
                   index=['K0', 'K1', 'K2', 'K3'])

df2 = pd.DataFrame({'C': ['C0', 'C1', 'C2', 'C3'],
                    'D': ['D0', 'D1', 'D2', 'D3']},
                   index=['K0', 'K1', 'K2', 'K3'])

df3 = pd.merge(df1, df2, left_index=True, right_index=True, how='inner')

print(df3)

     A   B   C   D
K0  A0  B0  C0  D0
K1  A1  B1  C1  D1
K2  A2  B2  C2  D2
K3  A3  B3  C3  D3


In [14]:
import pika
import json
from pika.credentials import PlainCredentials

# Replace these with your RabbitMQ credentials
username = 'admin'
password = 'admin'

credentials = PlainCredentials(username, password)
connection_parameters = pika.ConnectionParameters(host='rabbitmq.dcai2.local', credentials=credentials)

connection = pika.BlockingConnection(connection_parameters)
channel = connection.channel()

channel.queue_declare(queue='json_queue')

message = {
    'name': 'John Doe',
    'age': 30,
    'city': 'New York'
}

channel.basic_publish(exchange='',
                      routing_key='json_queue',
                      body=json.dumps(message))

print(" [x] Sent %r" % message)
connection.close()

 [x] Sent {'name': 'John Doe', 'age': 30, 'city': 'New York'}
